# Exercises

### 4. Fiddle with the activation functions. Try applying sigmoid transformation to both layers. The sigmoid activation is given by the method: tf.nn.sigmoid()

**Solution**

Find the part where we stack layers (Sequential()).

Adjust the activations from 'relu' to 'sigmoid'.
    
Generally, we should reach an inferior solution. That is because relu 'cleans' the noise in the data (think about it - if a value is negative, relu filters it out, while if it is positive, it takes it into account). For the MNIST dataset, we care only about the intensely black and white parts in the images of the digits, so such filtering proves beneficial.

The sigmoid does not filter the signals as well as relu, but still reaches a respectable result (around 95%).

**Try using softmax activations for all layers. How does the result change? Can you explain why that happens?**

# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf


import tensorflow_datasets as tfds


C:\Users\iolley2\Anaconda3\envs\py3-TF2.0\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

That's where we load and preprocess our data.

In [2]:


mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)


mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']


num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples, tf.int64)


num_test_samples = mnist_info.splits['test'].num_examples

num_test_samples = tf.cast(num_test_samples, tf.int64)



def scale(image, label):
    
    image = tf.cast(image, tf.float32)
    
    image /= 255.

    return image, label


scaled_train_and_validation_data = mnist_train.map(scale)


test_data = mnist_test.map(scale)


# let's also shuffle the data

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)


validation_data = shuffled_train_and_validation_data.take(num_validation_samples)


train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100


train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)


test_data = test_data.batch(num_test_samples)



validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model
When thinking about a deep learning algorithm, we mostly imagine building the model. So, let's do it :)

In [13]:
input_size = 784
output_size = 10

hidden_layer_size = 50
    
    
model = tf.keras.Sequential([
    
    
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    
    
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'), # 2nd hidden layer
    
    
    
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

### Choose the optimizer and the loss function

In [14]:


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
That's where we train the model we have built.

In [15]:


NUM_EPOCHS = 5


model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
540/540 - 4s - loss: 0.4655 - accuracy: 0.8665 - val_loss: 0.2456 - val_accuracy: 0.9280 - 4s/epoch - 7ms/step
Epoch 2/5
540/540 - 3s - loss: 0.2127 - accuracy: 0.9367 - val_loss: 0.1644 - val_accuracy: 0.9503 - 3s/epoch - 6ms/step
Epoch 3/5
540/540 - 3s - loss: 0.1522 - accuracy: 0.9540 - val_loss: 0.1323 - val_accuracy: 0.9600 - 3s/epoch - 6ms/step
Epoch 4/5
540/540 - 3s - loss: 0.1136 - accuracy: 0.9660 - val_loss: 0.0987 - val_accuracy: 0.9707 - 3s/epoch - 6ms/step
Epoch 5/5
540/540 - 3s - loss: 0.0895 - accuracy: 0.9728 - val_loss: 0.0769 - val_accuracy: 0.9790 - 3s/epoch - 6ms/step


## Test the model

As we discussed in the lectures, after training on the training data and validating on the validation data, we test the final prediction power of our model by running it on the test dataset that the algorithm has NEVER seen before.

It is very important to realize that fiddling with the hyperparameters overfits the validation dataset. 

The test is the absolute final instance. You should not test before you are completely done with adjusting your model.

If you adjust your model after testing, you will start overfitting the test dataset, which will defeat its purpose.

In [16]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 159ms/step - loss: 0.0939 - accuracy: 0.9716


In [17]:

print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.16%
